In [1]:
import os
import numpy as np
from PIL import Image
import torchvision
from torchvision import transforms
import torch
import pickle
import torch.nn as nn
from tqdm import tqdm
from datasets.robot_hand import RobotHandDataset

In [2]:
torch.cuda.is_available()

True

In [3]:
dataset = RobotHandDataset('train', '/mnt/d/data/robot_hand', mode='tail')
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1)

In [4]:
model = torchvision.models.resnet50()
model.conv1 = nn.Conv2d(10, 64, kernel_size=7, stride=2, padding=3,bias=False)
model.fc = nn.Linear(2048, 12)

print(len(data_loader))
for img, label in data_loader:
    print(img.shape)

    output = model(img)
    print(output)
    break

1019
torch.Size([1, 12, 224, 224])


RuntimeError: Given groups=1, weight of size [64, 10, 7, 7], expected input[1, 12, 224, 224] to have 10 channels, but got 12 channels instead

In [5]:
def train(logpath='', dataroot='/mnt/d/data/robot_hand', num_epochs=10):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    
    model = torchvision.models.resnet50()
    model.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3,bias=False)
    model.fc = nn.Linear(2048, 12)
    model.to(device)

    dataset = RobotHandDataset(split='train', dataroot=dataroot, mode='head')
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=4)
    dataset_eval = RobotHandDataset(split='train', dataroot=dataroot, mode='tail')
    data_loader_eval = torch.utils.data.DataLoader(dataset_eval, batch_size=4)
        
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(params, lr=0.01)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.5)

    len_dataloader = len(data_loader)

    for epoch in range(num_epochs):
        model.train()
        i = 0    
        epoch_loss = 0
        tqdm_bar = tqdm(data_loader, desc="Epoch {}/{}".format(epoch+1, num_epochs))
        for imgs, labels in tqdm_bar:
            i += 1
            # imgs = list(img.to(device) for img in imgs)
            # annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
            imgs = imgs.to(device)
            labels = labels.to(device) * 1000
            output = model(imgs)
            loss = nn.MSELoss()(output, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 
            tqdm_bar.set_description(f'Iteration: {i}/{len_dataloader}, Loss: {loss}')
            epoch_loss += loss
        
        # get eval set loss
        with torch.no_grad():
            model.eval()
            eval_loss = 0
            print('Evaluating...')
            for imgs, labels in data_loader_eval:
                imgs = imgs.to(device)
                labels = labels.to(device) * 1000
                output = model(imgs)
                loss = nn.MSELoss()(output, labels)
                eval_loss += loss
        
        scheduler.step()
        print(f'Epoch: {epoch+1}, Loss: {epoch_loss}, Eval Loss: {eval_loss}')

        torch.save(model.state_dict(), f'logs/{epoch+1}.pth')

train(num_epochs=50)

Iteration: 595/595, Loss: 0.15864497423171997: 100%|██████████| 595/595 [01:31<00:00,  6.50it/s]  


Evaluating...
Epoch: 1, Loss: 18694.013671875, Eval Loss: 94641.4296875


Iteration: 595/595, Loss: 0.16369278728961945: 100%|██████████| 595/595 [01:28<00:00,  6.71it/s]  


Evaluating...
Epoch: 2, Loss: 150.60545349121094, Eval Loss: 83723.578125


Iteration: 595/595, Loss: 0.11464127153158188: 100%|██████████| 595/595 [01:23<00:00,  7.15it/s]  


...

Evaluating...
Epoch: 39, Loss: 7.859512805938721, Eval Loss: 176.3374481201172


Iteration: 595/595, Loss: 0.0010940118227154016: 100%|██████████| 595/595 [01:21<00:00,  7.32it/s] 


Evaluating...
Epoch: 40, Loss: 7.994284152984619, Eval Loss: 146.49478149414062


Iteration: 61/595, Loss: 0.013846304267644882:  10%|█         | 61/595 [00:08<01:18,  6.81it/s]  


KeyboardInterrupt: 

In [6]:
import yaml

config_file = yaml.load(open('./experiments/hrnet/hrnet.yaml', 'r'), Loader=yaml.FullLoader)
config_file

{'experiment_name': 'hrnet',
 'model_name': 'hrnet',
 'batch_size': 24,
 'epoch': 210,
 'lr': 0.001,
 'lr_factor': 0.1,
 'lr_step': [170, 200],
 'resume': None}